# Jupyter Notebook approach for SCORE interaction


### Necessary packages 

In [ ]:
from checkscore.methods import readonly, external, payable
from checkscore.methods import Checkscore
from iconsdk.wallet.wallet import KeyWallet

password = input()
deployer_wallet = KeyWallet.load("keystore_ibriz.json",password)
deployer_wallet.get_address()

SCORE_ADDRESS = ""
Checkscore(SCORE_ADDRESS).fill_methods()

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
import requests
import json
import pickle
import csv
import os

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

mainnet= {
    "iconservice":"https://ctz.solidwallet.io",
    "nid" :1
    
}

yeouido={
     "iconservice":"https://bicon.net.solidwallet.io",
     "nid" : 3
    
}
euljiro={
     "iconservice":"https://test-ctz.solidwallet.io",
     "nid" : 2
    
}
pagoda={
    "iconservice":"https://zicon.net.solidwallet.io",
     "nid" :80
    
    
}
custom={
    "iconservice":"http://18.237.205.52:9000/",
     "nid":3
    
    
}


### Required service

In [3]:
env = custom #mainnet,yeouido,euljiro,pagoda,custom
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]



### A list for SCORE addresses 

In [4]:
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"

# Yeouido Testnet
# SCORE_ADDRESS = "cxf42190e91701bec907c7c4514eda6ee0af7f795b"
SCORE_ADDRESS = "cx19f99666108bab3db87d1c633c2bfb8d0b219433"


# Mainnet}
# SCORE_ADDRESS = "cx346a2c30ea2712557deb4d7392688750350d1dea"

### Importing necessary wallets

If you want to use your pre-exisiting wallets, execute the cell below:-

In [5]:
password = input()

deployer_wallet = KeyWallet.load("keystore_2",password)
deployer_wallet.get_address()


Liverpool@1


'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'

### OR creating new wallets 

If you want to use new wallets, execute the cell below:-

In [ ]:
deployer_wallet = KeyWallet.create()
print(deployer_wallet.get_address())
caller_wallet = KeyWallet.create()
print(caller_wallet.get_address())

### Loading ICX
Go to [ibriz faucet](https://icon-faucet.ibriz.ai/) to load ICX into your wallets.

### Deploying contract

In [ ]:
# params = {'_initialSupply':500000000, '_decimals': 18}
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('addressProvider'))\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

### Updating contract

In [6]:
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx162fe5b5d3276e9493de1fd440511f0844ca9a7c")\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('lendingPoolCore'))\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)


get_tx_result(tx_hash)

{'txHash': '0x973842e493f4432a6b7cf7fb678cd64ff86a943d318d4dd8d23d484595d5af0e',
 'blockHeight': 84833,
 'blockHash': '0x303ff61daf737f7ebd5a0a331bb9972f2a19c439137389c07b0f09c2caefe092',
 'txIndex': 1,
 'to': 'cx162fe5b5d3276e9493de1fd440511f0844ca9a7c',
 'stepUsed': 1801996000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1801996000,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

## Adding new reserve in LendingPoolCore



In [35]:
params ={"_reserve": {"reserveAddress":"cx072d8f95877a53df350e3dc3d8dba2f379037d42","oTokenAddress":"cx48dd26868c25cfa744aab9f2fccd76ed40c41454","totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"60","liquidationThreshold":"65","liquidationBonus":"10","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx162fe5b5d3276e9493de1fd440511f0844ca9a7c") \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx162fe5b5d3276e9493de1fd440511f0844ca9a7c', 'stepLimit': '0x58110', 'timestamp': '0x5b2e41b4b1fab', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'addReserveData', 'params': {'_reserve': {'reserveAddress': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42', 'oTokenAddress': 'cx48dd26868c25cfa744aab9f2fccd76ed40c41454', 'totalBorrows': '0', 'lastUpdateTimestamp': '0', 'liquidityRate': '0', 'borrowRate': '0', 'liquidityCumulativeIndex': '1000000000000000000', 'borrowCumulativeIndex': '1000000000000000000', 'baseLTVasCollateral': '60', 'liquidationThreshold': '65', 'liquidationBonus': '10', 'decimals': '18', 'borrowingEnabled': '1', 'usageAsCollateralEnabled': '1', 'isFreezed': '0', 'isActive': '1'}}}, 'signature': 'RBBL900UsBAzZq30dPV1kKEVRTM6EM8XBfzPpFzAA8EaR23dCt8/4eQNn04OYoedJNzXCw6GBL31AL4LFUZAEQA='}


{'txHash': '0x24d45a6584a2e31b7ed6c2c2d93729f13621a352aa1f2b069d5ee58095bf332f',
 'blockHeight': 16104,
 'blockHash': '0xac46e15f2c8347ebbff77c1c181f89040507cb0cbdc841e03bf34d8c2c696b94',
 'txIndex': 1,
 'to': 'cx162fe5b5d3276e9493de1fd440511f0844ca9a7c',
 'stepUsed': 260720,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 260720,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

## Setting reserve constants in LendingPoolCore

In [ ]:
params ={"_constants": [{"reserve":"cx072d8f95877a53df350e3dc3d8dba2f379037d42","optimalUtilizationRate":6*10**17,"baseBorrowRate":0,"slopeRate1":7*10**16,"slopeRate2":3} ]}
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx162fe5b5d3276e9493de1fd440511f0844ca9a7c") \
    .nid(NID) \
    .nonce(100) \
    .method("setReserveConstants")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

## Setting interfaces in LendingPool

In [ ]:
params ={"_address": "cx162fe5b5d3276e9493de1fd440511f0844ca9a7c" }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cxbf1a257dcb378b0fd41066fa87abef3f25992832") \
    .nid(NID) \
    .nonce(100) \
    .method("setLendingPoolCoreAddress")\
    .params(params)
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

In [ ]:
params ={"_address": "cx072d8f95877a53df350e3dc3d8dba2f379037d42" }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cxbf1a257dcb378b0fd41066fa87abef3f25992832") \
    .nid(NID) \
    .nonce(100) \
    .method("setUSDbAddress")\
    .params(params)
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

### Calling a readonly method

In [ ]:
params = {'_address': 'hxc9319d218ffff1672c9f1c15365c6f6c0d161c7b' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cxf479be3e8dd530fbb59a2bf58ce7a9903e68388e")\
    .method("get_last_ten_hands")\
    .params(params)\
    .build()
response = icon_service.call(_call)
print(response)
   

## Setting interfaces in LendingPoolDataProvider

In [ ]:
params ={"_address": "cx162fe5b5d3276e9493de1fd440511f0844ca9a7c" }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx84490f03cc04b799fe668545340eaeac7cae628c") \
    .nid(NID) \
    .nonce(100) \
    .method("setLendingPoolCoreAddress")\
    .params(params)
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

In [ ]:
params ={"_address": "cx293582520a75648d91a3c454ad9657aa4cbf1f7d" }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx84490f03cc04b799fe668545340eaeac7cae628c") \
    .nid(NID) \
    .nonce(100) \
    .method("setOracleAddress")\
    .params(params)
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)


## Setting Price in Oracle

In [ ]:
params ={"_base": "USDb", "_quote":"USD","_rate": 5 * 10**17 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx84490f03cc04b799fe668545340eaeac7cae628c") \
    .nid(NID) \
    .nonce(100) \
    .method("set_reference_data")\
    .params(params)
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

### Calling an external method

In [ ]:
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx62471f1ae14a5b243a04b22f4aa198e18112b2a6") \
    .nid(NID) \
    .nonce(100) \
    .method("toggle_game_status")\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

In [ ]:
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cxf479be3e8dd530fbb59a2bf58ce7a9903e68388e") \
    .nid(NID) \
    .nonce(100) \
    .method("set_global_cutcard")\
    .params({"_value":88})\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

### ICX Transfer

In [7]:
transaction = TransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("hx233633ad3d85fe2aed5ac71863ac5d7a2f31d16b")\
    .nid(3) \
    .nonce(100) \
    .value(10000*10**18)\
    .build()

estimate_step = icon_service.estimate_step(transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'hx233633ad3d85fe2aed5ac71863ac5d7a2f31d16b', 'stepLimit': '0x30d40', 'timestamp': '0x5b304321327fc', 'nid': '0x3', 'value': '0x21e19e0c9bab2400000', 'nonce': '0x64', 'signature': 'ruf71aLp9JAWD5V4Knr82AoJUlEnYj48NNtS7e0jPENbCItY/kJ48PF8OnGu6ylEVmzx96DGCerUwCUo+9MKOgA='}


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0xd4aac0a49123ec4effb8f2c4c5665bd4f7d6d6c8c13d62648e708b67d92d5874',
 'blockHeight': 84929,
 'blockHash': '0x4333c3bfcca54a798a95a7848d1afa97e267416782ee6d769c5ae19ca5bcecaa',
 'txIndex': 1,
 'to': 'hx233633ad3d85fe2aed5ac71863ac5d7a2f31d16b',
 'stepUsed': 100000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 100000,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [ ]:
balance = icon_service.get_balance("hxedcdfa97532eb690cf65c0f27ca1b5d4374b8122")
print(balance)

### Calling a payable method



In [ ]:

params={
        "_banker_bet_amt": 2*10**18,
        "_side_bet_type" :"PerfectPair",
        "_side_bet_amt": 10**18
        
     
    
}
# Skip side_bet_amount and side_bet_type if you dont want to play sidebet

call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx19f99666108bab3db87d1c633c2bfb8d0b219433")\
    .nid(NID)\
    .nonce(100)\
    .value(2*10**18)\
    .method("place_bet")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)


get_tx_result(tx_hash)

In [ ]:

params={
    '_data' : '{"reserve":"cxe4ced2048eab3fcd5c830058a21a32ffea8ea246","current_liquidity_rate": 5, "current_borrow_rate": 3}'
        }
    

# Skip side_bet_amount and side_bet_type if you dont want to play sidebet

call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cxe4ced2048eab3fcd5c830058a21a32ffea8ea246")\
    .nid(NID)\
    .nonce(100)\
    .method("set_reserve_data")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)


get_tx_result(tx_hash)

In [ ]:
cx4fb972950757f149a53f0e33c4eefac1feb77db4

In [ ]:

params={
   
    "_value": 20,
}
# Skip side_bet_amount and side_bet_type if you dont want to play sidebet

call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx4fb972950757f149a53f0e33c4eefac1feb77db4")\
    .nid(NID)\
    .nonce(100)\
    .method("set_global_cutcard")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)


get_tx_result(tx_hash)